In [ ]:
#!pip install pymongo
#!#pip install twitter
#!pip install tweepy

In [1]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser


In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secret']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secret']

mongod_connect = config['mymongo']['connection']

In [3]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection_job #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

In [ ]:
stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

In [ ]:
track = ['IA Job']


In [ ]:
class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track)

In [ ]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

In [ ]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "IA Jobs"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

In [4]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [5]:
count = 10 #number of returned tweets, default and max is 100
 # defin the location, in Harrisonburg, VA
q = "IA Jobs"  

In [6]:
search_results = rest_api.search.tweets( count=count,q=q)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

TwitterHTTPError: Twitter sent status 429 for URL: 1.1/search/tweets.json using parameters: (count=10&oauth_consumer_key=dTVLFrUodQQNF70Ae9FxXKsLl&oauth_nonce=17246981844949976472&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1572984832&oauth_token=1638198894-vd3DkvtvoBAhgouAjsZhc0bfHRpsHWDgbBotUJr&oauth_version=1.0&q=IA%20Jobs&oauth_signature=HtgCEpbXkH%2F3vxukRqabR%2Fs2Rss%3D)
details: {'errors': [{'message': 'Rate limit exceeded', 'code': 88}]}

In [ ]:

since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

In [7]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

1596
958


In [8]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "Ia Jobs"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: TMJ-USA PM Jobs
text: If you're looking for work in #Creston, IA, check out this job: https://t.co/ghtoVeFHO6 #ProductMgmt
----
name: Simone Marshall
text: Job opportunity: https://t.co/li6T42hj6d
----
name: सुनील शर्मा
text: RT @vivekagnihotri: Trending #swara_aunty is not the right thing to do. 

She is just doing her job professionally.
----
name: TMJ-VAH Jobs
text: We're hiring in #Harrisonburg, VA! Click the link in our bio to apply to this job and more: General Utility Worker… https://t.co/PVgbaDiXzz
----
name: Nancy Marquez
text: Job Opening for Medievalists: Department of English at James Madison University, tenure-track Assistant Professor i… https://t.co/nx3ssBHEfd
----
name: Erika Gaffney
text: Job opening: The Dept of English at @JMU is hiring a tenure-track Asst Professor in Medieval Studies, to begin Fall… https://t.co/jQmBgVzVEm
----
name: Brandy C. Williams
text: RT @egaffney4: Job opening: The Dept of English at @JMU is hiring a tenure-track Asst Professor i